# The intuition of cryptocurrencies, explained from the bottom up

**An exploration of how cryptocurrencies *really* work... by coding one from scratch.**

~

### by Darren Ang 
*Linkedin: Darren Ang  ||  Instagram: @whatislaweven  ||  YouTube: What Is Law Even*

~

## First, an introduction with some context!

While stuck in a short self-isolation period, I decided to finally try to understand how a cryptocurrency works. And there's no better way to understand how it works then to, well, *actually program it*. So here we are.

I was extremely inspired by the excellent explanation given in this 3Blue1Brown video: https://youtu.be/bBC-nXj3Ng4. For the most part, I'm following along with the video's structure, though I did check out a bunch of other videos along the way for more specific topics (namely, proof-of-work). 

The code isn't that great, but I think it works well enough to construct the intuition of crypto from the bottom up. The aim of this little project was to illuminate my own understanding of blockchains, and I hope it may illuminate yours as well; so to that end, I'd just like to emphasise that **you do not need to know how to code for this to make sense**. I hope I've written it clearly enough for those who haven't had my privilege of learning coding. You don't really need to look at the code blocks; it should be okay to focus on the outputs.

(Note: The code I'm displaying here is significantly reduced from the one I actually tinkered with. It omits some tangentially-important components of crypto systems, like the private and public key system, as well as user authentication. If you'd like to check that out, you can access my whole project at https://github.com/DarrenAngVGM/blockchain)

# The general theme: Who do you put your trust in?

The intuition of a crypto-'currency' builds off the idea of a 'currency'. I could include a whole essay on this, but I think this amazing YouTube video should suffice: https://youtu.be/S1Om2rSQcDE

Essentially, **'currencies' are built on trust.** For currencies tied to states, such as the US Dollar, we believe the money has value because of our trust in the financial institutions of that state.

Now, some people say that crypto is 'trustless'. If 'trustless' means you don't have to trust a central governing authority, then I'd say it's 99.9% correct. But if it means you don't have to trust anything at all, then it'd be flat-out wrong.

Because in a crypto system, you still need to 'trust' something. You need to trust the crypto system.

So here's how we'll construct the intuition of crypto: we'll take a journey that begins from trust in your friends, to trusting a central authority, to trusting a network of users, and finally, to trusting in computational effort. It might sound abstract now. I assure you it'll make more sense as we go along.

# Part 1: Trust in your friends.

Let's call on three good friends: Red, Blue, and Yellow. These three friends pay each other from time to time, but they're too lazy to settle in cash. So they just keep a mental note of how much they owe each other. 

Because this is a transaction between a closed network of friends, there's no real need for the currency to be in real money. So I'm going to go ahead and say that the three friends transact in **Awesome Coins, or ACs.** Makes no real difference anyway.

In [45]:
# This represents the friends and their account balance in the format {Name: Balance}
friends = {
    "Red": 0,
    "Blue": 0,
    "Yellow": 0
}

Let's now build a function to simulate transactions between friends.

In a transaction, one friend's (the payer's) account decreases by the payment amount, 
and the other friend's (the payee's) account increases by the same amount.

In [46]:
def transaction(users: dict, payer_name: str, payee_name: str, payment: float):
    users_copy = users.copy()
    users[payer_name] = users_copy[payer_name] - payment  # The payer loses the payment amount...
    users[payee_name] = users_copy[payee_name] + payment  # ... and the payee gains it

And now, let's simulate three transactions. We'll then check each friend's final account balance.

In [47]:
transaction(friends, "Red", "Yellow", 200)  # Red pays Yellow 200 ACs
transaction(friends, "Blue", "Yellow", 500)  # Blue pays Yellow 500 ACs
transaction(friends, "Red", "Blue", 300)  # Red pays Blue 300 ACs

print("Current account balances: ",friends)

Current account balances:  {'Red': -500, 'Blue': -200, 'Yellow': 700}


This system works as long as you remain friends. Because if I were Yellow here, I'd trust that Red and Blue would pay me the 500 and 200 ACs they owe me.

**In short, you need to trust your friends to pay you back.**

But this doesn't always make sense. If you're a commercial person doing commercial things, you wouldn't want to risk losing loads of your money based on the trust of your almost-stranger commercial partners. For that matter, you might get skittish once your partners start going negative, because it means *they might not be able to pay you back*.

So the utility of this system pretty much ends here. If you can't trust in your friends, then you might want to trust some central authority.

~

# Part 2: Trust in a central authority.

Let's bring in Red, Blue and Yellow again. Except, this time, they're not friends. They're just random people who decide to transact with each other. They could run away with other peoples' money.

Because they want to make sure they'll get paid back, they decide on a rule: **each of them will open an account with a bank, with 1000 ACs inside the account before they even start transacting**. And once they go below the limit, their transactions will be automatically rejected.

This is pretty easy to implement in code. Here's what it might look like...

In [48]:
users = {
    "Red": 1000,
    "Blue": 1000,
    "Yellow": 1000
}


def transaction(users: dict, payer_name: str, payee_name: str, payment: float):
    users_copy = users.copy()
    payer_balance = users_copy[payer_name] - payment
    
    # Just one difference: We'll reject the transaction if the payer's account balance goes negative.
    if payer_balance < 0:
        print(f"Transaction of {payment} from {payer_name} to {payee_name} rejected due to insufficient funds.")
    else:
        users[payer_name] = payer_balance  
        users[payee_name] = users_copy[payee_name] + payment  
        print(f"Transaction of {payment} from {payer_name} to {payee_name} successful.")

# Normal transactions would still go through...
transaction(users, "Red", "Yellow", 500)
transaction(users, "Blue", "Yellow", 200)

# But any transaction which goes over the account balance would be rejected.
transaction(users, "Red", "Yellow", 680.42)

print("Current account balances: ", users)

Transaction of 500 from Red to Yellow successful.
Transaction of 200 from Blue to Yellow successful.
Transaction of 680.42 from Red to Yellow rejected due to insufficient funds.
Current account balances:  {'Red': 500, 'Blue': 800, 'Yellow': 1700}


### Cool, but doesn't this put too much power in the central authority?

Well yes, but it's also pretty close to how a modern bank works.

Of course, a modern bank wouldn't be manually keying in every transaction. There will be mechanisms to ensure that the payers actually are who they claim to be ('authentication'), and most transactions can be initiated by the payers themselves. The banks just have the technical capabilities to make the numbers move.

I had implemented some of these mechanisms in my actual code. But I didn't think it'd be that relevant here. Because **the problem remains: you're still putting your trust in some central authority.** Nothing stops them, or some crafty hackers, from keying in random transactions...

In [49]:
transaction(users, "Yellow", "Blue", 999.123456789)

print(users)

Transaction of 999.123456789 from Yellow to Blue successful.
{'Red': 500, 'Blue': 1799.123456789, 'Yellow': 700.876543211}


...it was *that* easy for me to just play with the values. Of course, it's not really that simple in real life, and real banks have attracted distrust for different reasons. But still, you need to trust me, the bank.

**Is there any system that doesn't involve trusting in a central authority of any sort?**

~

# Part 3: Trust in the network of users.

We now move one degree away from a central authority. Cryptocurrencies are 'decentralised' systems, and you might have heard that it's secured on a 'network of users'. Or, some call it a 'distributed ledger'. That's pretty complicated, so let's take it step by step.

## (1) The ledger: A record of transactions

So far, we haven't cared much about our transaction history. As the basic idea of a crypto coin is built upon this, we'll need to start caring now.

Let's return to our three users. I'll illustrate a record of transactions between them on a table. If the code looks confusing, don't worry; the only important part is the table below it. There, the users are represented as columns, and the individual transactions are represented as rows.

In [50]:
import pandas as pd

df = pd.DataFrame.from_dict({"Red": [1000, 0, -400, 300], "Blue": [1000, 420, 400, 0], "Yellow": 
                             [1000, -420, 0, -300]})
print(df)

    Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300


At this stage, we could also implement the check that ensures account balances don't go below zero. It's not that relevant here, so I'll leave that out.

But anyway, the ledger should speak for itself. In summary, all users start with 1000 ACs, and then:
1. Yellow pays 420 ACs to Blue;
2. Red pays 400 ACs to Blue;
3. Yellow pays 300 ACs to Red.

## (2) The 'distributed' ledger and 'consensus mechanisms'

Alright, so we have a ledger. Still seems like a bank.

### How about a 'distributed' ledger?

So, enter one of the key innovations of cryptocurrencies: the 'distributed ledger' system. In essence, **every user gets a copy of the ledger**. In our example, that's Red, Blue, and Yellow.

Problem is, when everyone has a copy of the ledger, everyone has the power to tamper with it. For example, if Blue wanted to get paid a lot more, they could do this (again, just focus on the table below the code):

In [51]:
red_ledger = df.copy()
yellow_ledger = df.copy()

blue_ledger = pd.DataFrame.from_dict({"Red": [1000, 0, -1000, 0], "Blue": [1000, 700, 1000, 300], "Yellow": 
                             [1000, -700, 0, -300]})

print(f"Red's Ledger:\n{red_ledger}\n")
print(f"Blue's Ledger:\n{blue_ledger}\n")
print(f"Yellow's Ledger:\n{yellow_ledger}\n")

Red's Ledger:
    Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300

Blue's Ledger:
    Red  Blue  Yellow
0  1000  1000    1000
1     0   700    -700
2 -1000  1000       0
3     0   300    -300

Yellow's Ledger:
    Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300



### And a 'consensus mechanism'!

Of course, that'd be a silly situation. And of course, there's a simple way to prevent this from happening: implementing a **consensus mechanism**. That is, if more than 50% of users in the network agree that a version of the ledger is the genuine one, then that version is taken to be genuine indeed.

In the above case, Red and Yellow have the same ledger, so that makes up for 67% of the users, which means that Blue's ledger must have been tampered. Blue's ledger would probably get rejected.

**What this really means is, your trust is in the consensus built by the network of users.**

But we recall that 'consensus' means '51% of users agree'. Sure, it might seem pretty hard to control 51% of the users in the network. But all you need is one split-second of that control; as you'll see later on, computers can work that fast and even faster. Some call this a **'51% attack'**. 

So as a whole, trusing in the network of users is quite secure, but it'll still be possible to tamper with the ledgers. That is, to initiate 51% attacks.

Therefore, we need a way to make it *obscenely* hard to tamper with the ledgers. And that's where the hype really begins.

~

# Part 4: Trust in computational effort—the blockchain and proof-of-work

Enter the blockchain. It's got a variety of other applications, but in essence, it's some record of transactions (or any actions, really) that is very hard to mess with. This is usually where it gets really confusing; again, we'll take it step by step.

## (1) Transactions as 'blocks': the power of cryptographic hash functions

For the sake of simplicity, let's say that each transaction counts as one 'block'. That's not how it works in reality; usually, one block consists of a bunch of transactions. But it suffices for this illustration.

### How 'cryptographic hash functions' work

Now, for the magic of crypto coins to work, we'll need to run the data of each block through a **'cryptographic hash function'**. I'd explain it, but it's probably easier to demonstrate how it works through the industry-standard hash, SHA256 (Secure Hash Algorithm 256-bit). Again, don't worry if the code looks confusing; the point still comes across if you focus on the output below it.

In [52]:
import hashlib

sha = hashlib.new("sha256")
sha.update(b"wassup_yall")  # This runs the string of text "wassup_yall" (in bits) into the hash function

print(sha.hexdigest())  # This produces... the monstrosity below.

ba7672e97fb2aa5ff3fbcb3bef3aee96002540884886d6670f4622647e4995da


That looks like gibberish. And it turns out that, for anything I throw into the hash function, gibberish comes out.

In [53]:
sha = hashlib.new("sha256")
sha.update(b"123456789")

print(sha.hexdigest())

15e2b0d3c33891ebb0f1ef609ec419420c20e320ce94c65fbc8c3312448eb225


In [54]:
sha = hashlib.new("sha256")
sha.update(b"hey i saw a cool thing today!!")

print(sha.hexdigest())

a67c803fae7da75e170b9f0965ec889d98903f57a47f3ebc0149f08a162780ce


In [55]:
sha = hashlib.new("sha256")
sha.update(b"jasfkhgjbsfdhsgdskbsndhgkdjabkihugrwtbjewdhsigdbrehfbhsvdkjhi")

print(sha.hexdigest())

7913a53e89c13227897b3e902ef45f11bf1061a2c24596b250a84a5566deaa7c


Still gibberish. But here's the catch. Even though the outputs might seem random, they are not. **For any given input data, there will be the same output hash.** 

Let's demonstrate that with the initial 'wassup_yall' example...

In [56]:
sha = hashlib.new("sha256")
sha.update(b"wassup_yall")

print(sha.hexdigest()) 

ba7672e97fb2aa5ff3fbcb3bef3aee96002540884886d6670f4622647e4995da


If you scroll up the page a bit, you'll notice that it's exactly the same as our earlier 'wassup_yall' example! And I can't even change the input by a single character, because **every small change in the input data results in a huge change to the output hash.** You can see that in action below.

In [57]:
sha = hashlib.new("sha256")
sha.update(b"wassup yall")  # It's the same, but without the underscore!

print(sha.hexdigest()) 

ad4fb8bb7c96a4c9864379a0dff2315865423f2c383fd255b90b0229a01f4152


### That's cool, but why would I want to use a hash function like SHA256? 

The magic of a hash is two-fold. First, it's very easy to verify if your input data is legitimate. Think of our earlier 'wassup_yall' example: the only way I could get the hash output of 'ba7672e97fb2aa5ff3fbcb3bef3aee96002540884886d6670f4622647e4995da' is by keying in 'wassup_yall'. Exactly. Even 'wassup yall' without the underscore wasn't okay. So if you keyed in input data the first time and got the hash, and you keyed in something to get the hash again, I can be pretty sure you keyed in the same thing.

But the flip-side to that is that, even if I know you keyed in the same input, I can't easily determine the exact data you inputted. I mean, could you identify 'wassup_yall' from the string 'ba7672e97fb2aa5ff3fbcb3bef3aee96002540884886d6670f4622647e4995da'?. Probably not. Which means, **if you were given a hash output and asked to find the input data, your best option is to guess until you get it right.**

Therefore, we call SHA256 a 'cryptographic hash function', and say that it only operates in one-way. Apparently this has never been tested, but I'd love to see someone try.

### So what is the purpose of a cryptographic hash function in a cryptocurrency? 

Well, in order to move beyond trust in the network of users, we need a way to ensure that genuine ledgers get accepted, while tampered ledgers get rejected. This has to be entirely outside of the users' control.

The cryptographic hash function ensures this in really complicated ways, and the prevailing method is that of 'proof-of-work'. We'll get to that in a bit.

But for now, we need to explain how we get from the ledger to hash outputs.

### Hashing the data in 'blocks' on the ledger

We'll start by breaking up the ledger into **'blocks' of data**. Each 'block' will then be fed into the cryptographic hash function. This means that any tampered versions of that same block will yield a completely different output hash.

As we mentioned above, we'll treat each transaction as its own 'block' for the purposes of this explanation. 

At this point, it's easier to use the code to demonstrate how the 'blocks' get hashed, and why that's useful. I'll just cook up a simple hash function that runs two values through SHA256: the transaction number and the payment amount. Again, the only important part is the output hash, below the code.

In [58]:
def single_block_hash(transaction_no, payment):
    sha = hashlib.new("sha256")
    to_hash = transaction_no * payment  # This just multiplies the two values together
    
    sha.update(str(to_hash).encode("ascii"))  # This turns the result into bits and hashes it
    return sha.hexdigest()

# Let's take transaction 1, of 420 ACs from Blue to Yellow
t1_hash = single_block_hash(1, 420)
print(t1_hash)

db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d


That string of random characters represents transaction 1, of 420 ACs from Blue to Yellow. 

We can do the same thing for transactions 2 and 3, to generate a sequence of hashes that represents the entire ledger of transactions...

In [59]:
t2_hash = single_block_hash(2, 400)
t3_hash = single_block_hash(3, 300)

print("Transaction 1:", t1_hash, "\nTransaction 2:", t2_hash, "\nTransaction 3:", t3_hash)

Transaction 1: db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d 
Transaction 2: 1a1cf797fabe7f95836fabeca626907c77b3e6c9aff7c2290b396a238c69362e 
Transaction 3: bdc5d8a48c23897906b09a9a3680bd2e9c8b3121edbda36f949800f0959c8d55


And as we recall, if even a small detail is changed, the entire hash changes. We can demonstrate this by adding 0.00000001 ACs to the payment in Transaction 2.

In [60]:
t2_hash = single_block_hash(2, 400.00000001)

print("Transaction 1:", t1_hash, "\nTransaction 2:", t2_hash, "\nTransaction 3:", t3_hash)

Transaction 1: db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d 
Transaction 2: f3483b9638552f9e7d1724ddb80c674c4d78828a72286dc9dcc0f1af15419bfb 
Transaction 3: bdc5d8a48c23897906b09a9a3680bd2e9c8b3121edbda36f949800f0959c8d55


In the above case, you can see that adding a mere 0.00000001 ACs to the transaction changed the whole of transaction 2's hash. This signals some kind of tampering.

So now we have a way to detect tampering of single transactions. But just by adding *one more element* to the mix, we can make it significantly harder to tamper entire chains of transactions.

## (2) Chaining up the blocks; making them really hard to alter

That single variable we need to add to our hash function, to make entire chains of transactions much harder to tamper with, is **the hash of the previous block**. That's why we call it a **'blockchain'**: the hashes of the previous blocks 'chain' up the sequence of transactions together. 

Here's how that might look like in code; you can just skip to the output below, for the sequence of hashes that represents transactions 1 to 3.

In [61]:
def blockchain_hash(transaction_no, payment, prev_hash):
    """I'll take the characters of the previous hash, convert them into numbers, then add them up. 
    Then, I'll multiply that sum of digits with the transaction number and payment amount.
    That should make for a pretty uncrackable blockchain."""
    
    char_values = {"a": 11, "b": 12, "c": 13, "d": 14, "e": 15, "f": 16}
    sum_chars = 0
    
    if prev_hash == 1:  # I'll treat the first block's ('genesis block') hash as 1, there's a special operation.
        sum_chars = 1
    else:
        for each_char in str(prev_hash):
            if each_char in char_values.keys():
                digit = char_values[each_char]
            else:
                digit = int(each_char)
            sum_chars += digit
    
    sha = hashlib.new("sha256")
    to_hash = transaction_no * payment * sum_chars
    
    sha.update(str(to_hash).encode("ascii"))
    return sha.hexdigest()

t1_hash = blockchain_hash(1, 420, 1)  
t2_hash = blockchain_hash(2, 400, t1_hash)
t3_hash = blockchain_hash(3, 300, t2_hash)

print("Transaction 1: ", t1_hash, "\nTransaction 2: ", t2_hash, "\nTransaction 3: ", t3_hash)

Transaction 1:  db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d 
Transaction 2:  4551038f9e2a01f0a92c501849a5c5e0057b21cfce5654d9657dd5d3132b7505 
Transaction 3:  4e1c235628476d557877c5825a46e0f5fbbf70a6a24c61f51a34efedc0d3e5dc


That looks really complicated, but more importantly, **watch what happens when I change just a tiny detail in the very first block...** (from 420 to 420.68 in transaction 1)

In [62]:
t1_hash = blockchain_hash(1, 420.68, 1)  
t2_hash = blockchain_hash(2, 400, t1_hash)
t3_hash = blockchain_hash(3, 300, t2_hash)

print("Transaction 1: ", t1_hash, "\nTransaction 2: ", t2_hash, "\nTransaction 3: ", t3_hash)

Transaction 1:  6315a3f36a6ddb0d7796992f012a6a7e898b0c3d5d6aea100842e860f54b7883 
Transaction 2:  563f229596803ff988c3773fa8a15093e4607793164b0a47250300117015641c 
Transaction 3:  bb2c716bb2062143d93b0949d81855b17d6eab654b37352eb4235d2b0729a820


The whole sequence of hashes changes. And if I change the second transaction...

In [63]:
t1_hash = blockchain_hash(1, 420, 1)  
t2_hash = blockchain_hash(2, 400.042, t1_hash)
t3_hash = blockchain_hash(3, 300, t2_hash)

print("Transaction 1: ", t1_hash, "\nTransaction 2: ", t2_hash, "\nTransaction 3: ", t3_hash)

Transaction 1:  db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d 
Transaction 2:  c16fb2f38346efba5f9941f5ffe3fc5f77a08d182f34a6a13ad7106cd8bfb0b0 
Transaction 3:  8d17031e7a7a959fab6cb07968df47c29983c26587d6972a021ed6c26afe624c


... the first hash stays the same, but **every block after the tampered block will have a different hash** when compared to the genuine series of transactions (or, the genuine blockchain).

What this means is, if you try to enter a fake transaction into your ledger, this will not just result in a different hash for that transaction. **Any tampering of a transaction will result in all subsequent transactions having different hashes.** This will eventually become really obvious to the users who are in control of the genuine ledgers.

For example, here's a function that goes through the ledger and generates its blockchain (that is, its sequence of hashes). The output below represents the sequence of hashes of the transactions; the first block is just given a hash of 1. 

We'll bring back the earlier example of Blue's tampered ledger, as compared to Red and Yellow's genuine ledgers. It should be really obvious that Blue's ledger has a different sequence of hashes than Red and Yellow's.

In [64]:
def blockchain_from_ledger(ledger):
    """Takes a ledger (df), goes through the rows and generates the blockchain for the whole ledger as a list."""
    
    blockchain = [1] # Previous hash is 1 for first transaction, otherwise it's the previous hash
    for index, row in ledger.iterrows():
        
        if index > 0:  # Skips genesis block
            transaction_no = index
            
            for index, value in row.items():  # Finds positive value for each row; that's the payment amount
                if value > 0:
                    payment = value
            
            prev_hash = blockchain[-1]
            
            current_hash = blockchain_hash(transaction_no, payment, prev_hash)
            blockchain.append(current_hash)
            
    return blockchain
                
red_blockchain = blockchain_from_ledger(red_ledger)
blue_blockchain = blockchain_from_ledger(blue_ledger)
yellow_blockchain = blockchain_from_ledger(yellow_ledger)

print("Red's blockchain:\n", red_blockchain, "\n")
print("Blue's blockchain:\n", blue_blockchain, "\n")
print("Yellow's blockchain:\n", yellow_blockchain, "\n")

Red's blockchain:
 [1, 'db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d', '4551038f9e2a01f0a92c501849a5c5e0057b21cfce5654d9657dd5d3132b7505', '4e1c235628476d557877c5825a46e0f5fbbf70a6a24c61f51a34efedc0d3e5dc'] 

Blue's blockchain:
 [1, '99ee50221221864d50c60baea6f14d8ac2e235cc6e78be6088cd40cc97fca394', '29214d31fe188ceb9bce1cccc61c7046da045c99e2780135209cdf0062cb3070', '8f31042744f16a3b40b663cfe54736f1eab67d3b531d69aa1bbba44de62cc6cd'] 

Yellow's blockchain:
 [1, 'db55da3fc3098e9c42311c6013304ff36b19ef73d12ea932054b5ad51df4f49d', '4551038f9e2a01f0a92c501849a5c5e0057b21cfce5654d9657dd5d3132b7505', '4e1c235628476d557877c5825a46e0f5fbbf70a6a24c61f51a34efedc0d3e5dc'] 



**As a result of the tampering, Blue's blockchain looks completely different from Red's and Yellow's.** That means it will be rejected by the consensus mechanism. Easy enough.

But there's one last piece of the puzzle left. If you copy this code and try running it in your own Python console, you'll realise that it all runs extremely quickly. Like, crazily quickly. 

This gives fraudsters like Blue an opportunity to seize control over the required 51% of the ledgers; to do a '51% attack'. They could just borrow a whole bunch of computers and try to send the transaction through; it'll only take a split-second.

That's why we need to slow people like Blue down, and trust that there's no opportunity for these kinds of ledger attacks to happen. That's the place of the proof-of-work concept. And that's the final piece of our puzzle, at least for today.

## (3) Proof-of-work: Basically, slowing things down.

So, we know that SHA256 hashes basically can't be cracked using math hacks. **If you get a SHA256 puzzle, the best way to answer it is to guess until you get it right.** This should slow down any attempts to gain control over 51% of the ledgers.

Okay, then. Let's make a SHA256 puzzle. And let's make it such that, for a 'block' (in our case, one transaction) to be added to everyone's ledgers, it must solve the puzzle.

### Constructing a SHA256 puzzle!

Bitcoin's surprisingly simple puzzle runs like this:
1. Take the block's data and combine it with the guessed number (a 'number only used once', or a 'nonce');
2. When the combined data is hashed through SHA256, the result must begin with 30 zeroes (this number can be changed).

We'll modify it for our own cryptocurrency as follows:
1. Take the block's hash and sum up its digits, then multiply that by the guessed nonce;
2. When the product of the block's digits and the nonce is hashed through SHA256, the result must begin with 2 zeroes.

Here's how that puzzle will look in code, if you're interested:

In [65]:
def proof_of_work(transaction_data: str, nonce: float, difficulty: int):
    """The puzzle! Returns True if the required difficulty (number of zeroes required) is achieved, False otherwise.
    1. Takes the transaction data as a hexadecimal string and sums up the digits, then multiplies that by the nonce.
    2. Then, the product is run through SHA256 and checked for the number of required zeroes."""
    
    # This sums up the digits of the transaction hash; converts hexadecimal letters to corresponding numbers.
    conversion_dict = {"a": 11, "b": 12, "c": 13, "d": 14, "e": 15, "f": 16}
    sum_digits = 0
    for char in str(transaction_data):
        if char in conversion_dict.keys():
            sum_digits += conversion_dict[char]
        else:
            sum_digits += int(char)
    
    num_to_hash = sum_digits * nonce  # This obtains the product of the digit-sum and the nonce
    
    sha = hashlib.new("sha256")
    sha.update(str(num_to_hash).encode("ascii")) # This hashes the product 
    
    solution = "".join(["0" for i in range(difficulty)])  # This generates the number of zeroes required
    
    # This compares the hash output's first few characters with the solution, and returns True if it matches
    if sha.hexdigest()[:difficulty] == solution:  
        print(f"\nNonce for difficulty of {difficulty} zero bits found.\n"
              f"SHA256 hash is {sha.hexdigest()}")
        return True
    else:
        return False

So let's take stock. **We have a puzzle that basically requires users to guess numbers (nonces) until they get the right one.**

That sounds like an insurmountable challenge, but you'll be surprised at how fast computers can do this guessing game. Here's a function that does nothing but plug in random decimals (floats) into the proof-of-work puzzle until they get it right...

In [66]:
from timeit import default_timer
from random import random

def miner(transaction_data, difficulty):
    """It sounds cool, but it's actually really dumb. Guesses random floats until one answers the SHA256 puzzle.
    Also times how long it takes to reach the solution, and counts how many guesses were made."""
    
    guess_counter = 0
    start_time = default_timer()
    
    # The miner guesses a random float and plugs it into the proof-of-work puzzle.
    # If it doesn't succeed, it tries again.
    while True:
        nonce = random()
        if proof_of_work(transaction_data, nonce, difficulty) is True:
            end_time = default_timer()
            print(f"\nTime taken: {end_time - start_time}s\t||\tNumber of guesses: {guess_counter}")
            return True
        else:
            guess_counter += 1

Now that we have our puzzle, and a bot to brute force guesses until the puzzle is solved (called a **'miner'**), it's time to try it on an actual transaction! 

We'll go back to Red's (genuine) ledger from above, and add one transaction to it. But this time, we'll require a proof of work to be done, in order to slow down anyone trying to control 51% of the chain.

There's some code to be written for this new style of transaction that requires proof of work. It looks something like this; it's the most complicated function we've done in this demonstration, so again, no need to understand how it works exactly.

In [67]:
def add_transaction_to_ledger(ledger, payer_name, payee_name, payment, difficulty):
    
    users = ledger.columns
    payer_loc, payee_loc = None, None  # Initialises variables representing the location of the payer and payee.
    
    # Finds column no. for payer and payee.
    column_count = 0  
    for user in users:
        if user == payer_name:
            payer_loc = column_count
        elif user == payee_name:
            payee_loc = column_count
        column_count += 1

    # Assume payer and payee will be found.
    # Form a new row, which will be added to the ledger once proof-of-work is done.
    new_row = list()
    for i in range(len(users)):
        if i == payer_loc:
            new_row.append(-payment)
        elif i == payee_loc:
            new_row.append(payment)
        else:
            new_row.append(0)
    
    # Generates the blockchain representing the genuine ledger, obtains the hash of the previous block
    blockchain = blockchain_from_ledger(ledger)
    prev_hash = blockchain[-1]
    
    # Finds the hash of the new block
    transaction_no = len(ledger.index)
    new_block_hash = blockchain_hash(transaction_no, payment, prev_hash)
    
    # Calls on a miner to brute force guess the SHA256 puzzle until it is solved
    miner(new_block_hash, difficulty)
    
    # Once the miner is done, then the transaction can proceed
    ledger.loc[transaction_no] = new_row
    print(f"Transaction no {transaction_no} successful: {payment} ACs from {payer_name} to {payee_name}")

Whoa, that's a handful. Because it's a pretty big chunk of code, I'll explain what it does in some bullet points. It's about as close as I'd get to coding an actual cryptocurrency's proof-of-work puzzle.

Now, the function above requires some information to work: the ledger, the payer, payee and payment, as well as the difficulty (the number of zeroes required). This information is entered into the function in order, with commas in between.

*After the information is added in, the function operates in these steps:*
1. The ledger is scanned, to identify the locations of the payer and payee;
2. A new row (new transaction) is generated, where the payer's column reflects a decrease in the payment amount and the payee's column reflects an increase;
3. The blockchain representing the transactions on the ledger is generated;
4. The hash of the last block in the chain is stored (recall that this 'chains' the 'blocks' together);
5. The hash of the new block (the new transaction) is calculated, based on the required information and the last block's hash;
6. A miner bot is called to solve the SHA256 puzzle through brute force guesses (this slows the system down to prevent any 51% takeovers);
7. Once the miner is done, the transaction proceeds and is recorded in the ledger.

If that was a lot to take in, it's fine, because there's no real need to take it all in! I mean, what better way is there to illustrate how this works than to execute it?

**We'll deploy this function on a new transaction of 100 ACs between Red and Yellow.** You'll notice that the miner will report how many guesses, and how long it took, to brute force the SHA256 puzzle. And once the proof-of-work is done, the ledger will be updated to reflect the new transaction.

In [69]:
# Recall: the 2 at the end of this function represents the number of zeroes required!
add_transaction_to_ledger(red_ledger, "Red", "Yellow", 100, 2)

print("\nCurrent genuine ledger:\n", red_ledger)


Nonce for difficulty of 2 zero bits found.
SHA256 hash is 0072f8392c3aa670fd874f4f68407b218094ae35e848fd65f0a7e7ca30c71a19

Time taken: 0.015730167000583606s	||	Number of guesses: 430
Transaction no 4 successful: 100 ACs from Red to Yellow

Current genuine ledger:
     Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300
4  -100     0     100


... wait, how long did that take? It'll be different every time it's run, **but it's certainly way too fast!** Turns out that the computer could brute force its way to a two-zero solution in split-seconds. 

Let's try it again to make sure it's not a fluke, this time for a transaction of 25 ACs between Blue and Yellow...

In [70]:
add_transaction_to_ledger(red_ledger, "Blue", "Yellow", 25, 2)

print("\nCurrent genuine ledger:\n", red_ledger)


Nonce for difficulty of 2 zero bits found.
SHA256 hash is 00c410aa35615884816e3fc5c04478f3895517636f98540e74300fb203f36281

Time taken: 0.0014761669999643345s	||	Number of guesses: 49
Transaction no 5 successful: 25 ACs from Blue to Yellow

Current genuine ledger:
     Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300
4  -100     0     100
5     0   -25      25


I don't know about you, but for me, it was even faster than the last time: the best numbers I saw were 49 guesses in 0.001 seconds. That doesn't do anything to slow down the crooks who want to do a '51% attack' and control 51% of the ledgers.

What can we do to stop this? Well, the answer is quite simple: we can just increase the number of zeroes required. That is, **we can increase the difficulty of the puzzle.**

### Increasing the difficulty of the proof-of-work puzzle: As easy as adding zeroes!

At this point, I'd type in some other fun text, but I think I'll let the numbers speak for themselves. I'll simulate four random transactions below, for 3, 4, 5 and 6 zeroes respectively.

In [71]:
# Difficulty: 3 zeroes
add_transaction_to_ledger(red_ledger, "Red", "Yellow", 25, 3)

print("\nCurrent genuine ledger:\n", red_ledger)


Nonce for difficulty of 3 zero bits found.
SHA256 hash is 00013df78d86318a12e1480aa8643318e9acfc9994c9ac456bbae5d2b71f6b8b

Time taken: 0.19770462499946007s	||	Number of guesses: 18104
Transaction no 6 successful: 25 ACs from Red to Yellow

Current genuine ledger:
     Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300
4  -100     0     100
5     0   -25      25
6   -25     0      25


In [72]:
# Difficulty: 4 zeroes
add_transaction_to_ledger(red_ledger, "Yellow", "Blue", 75, 4)

print("\nCurrent genuine ledger:\n", red_ledger)


Nonce for difficulty of 4 zero bits found.
SHA256 hash is 00006e58fac0d91316ae4d33aeb1eee0973e4b5403a38b57eebb88265455b65c

Time taken: 0.26057733299967367s	||	Number of guesses: 26405
Transaction no 7 successful: 75 ACs from Yellow to Blue

Current genuine ledger:
     Red  Blue  Yellow
0  1000  1000    1000
1     0   420    -420
2  -400   400       0
3   300     0    -300
4  -100     0     100
5     0   -25      25
6   -25     0      25
7     0    75     -75


In [73]:
# Difficulty: 5 zeroes
add_transaction_to_ledger(red_ledger, "Red", "Blue", 68.42, 5)

print("\nCurrent genuine ledger:\n", red_ledger)


Nonce for difficulty of 5 zero bits found.
SHA256 hash is 00000b962a5ad83a88d82e48f4f07c8c01e8f91ffcddb90d6de91141e6b1a6f8

Time taken: 1.1631290829991485s	||	Number of guesses: 132486
Transaction no 8 successful: 68.42 ACs from Red to Blue

Current genuine ledger:
        Red     Blue  Yellow
0  1000.00  1000.00  1000.0
1     0.00   420.00  -420.0
2  -400.00   400.00     0.0
3   300.00     0.00  -300.0
4  -100.00     0.00   100.0
5     0.00   -25.00    25.0
6   -25.00     0.00    25.0
7     0.00    75.00   -75.0
8   -68.42    68.42     0.0


In [74]:
# Difficulty: 6 zeroes
add_transaction_to_ledger(red_ledger, "Blue", "Red", 420.68, 6)

print("\nCurrent genuine ledger:\n", red_ledger)


Nonce for difficulty of 6 zero bits found.
SHA256 hash is 00000027ec8f52a25a57162f70d90beae2e0622a72d0ed068a05ce3f2613f245

Time taken: 50.05204829200011s	||	Number of guesses: 6017888
Transaction no 9 successful: 420.68 ACs from Blue to Red

Current genuine ledger:
        Red     Blue  Yellow
0  1000.00  1000.00  1000.0
1     0.00   420.00  -420.0
2  -400.00   400.00     0.0
3   300.00     0.00  -300.0
4  -100.00     0.00   100.0
5     0.00   -25.00    25.0
6   -25.00     0.00    25.0
7     0.00    75.00   -75.0
8   -68.42    68.42     0.0
9   420.68  -420.68     0.0


The timings and number of guesses required will change every time, so it's best if you tried to play with this yourself on your own Python console. But what's really clear to me is, **as I increase the difficulty of the SHA256 puzzle, the amount of time required to brute force guess the nonce increases exponentially.**

So that's all you really need to do to slow down the 51% attackers! If they want to gain 51% control over the ledgers, they also need to perform the guesswork required to verify those fraudulent transactions. Some people have crunched the numbers for the amount of computational power (i.e. computers, electricity, etc.) you'd need to do that, and it's astronomical.

Basically, **the proof-of-work mechanism makes it practically impossible to control 51% of the ledgers.** Which means that the chance for someone to key in fraudulent transactions is basically zero.

Interestingly, doing this also allows Bitcoin to control the supply of their currency; as the amount of cryptocurrency increases, the difficulty can be quite easily increased by changing the number of zeroes required. It's probably more robust than just changing zeroes, but hey, the zeroes system works. 

# Conclusion: Cryptocurrency really is smart, and secure, and awesome... 

I hope this experiment has been meaningful to you, and I hope my little explanations have been helpful! The ultimate conclusion of this journey for me, from trust in your friends to trust in computational power, is that **the technology really is amazing.**

It was easy to implement, the SHA256 puzzles worked really well, and honestly, I think I have a better understanding of why people think these cryptocurrencies aren't just a hoax. There's good reason for the hype around cryptocurrencies to exist: the technology really is quite amazing.

# ... but we shouldn't view it in isolation of the real world.

But of course, that's the technology in itself. In reality, cryptocurrencies are (currently) extremely volatile investments that haven't been well-received by most governments. There are a bunch of broader reasons for that: their security lets them be used for crime financing, their ease of creation makes it such that there's loads of cryptocurrencies competing with one another, and their freedom from government control also means that it's really hard for governments to deal with crypto frauds. Not to forget the damage to the environment caused by crypto mining.

In short, having done this experiment, I do have quite a lot of musings on crypto more broadly. I'll be doing a post on my personal law blog regarding that in a bit.

But yes, the technology is awesome. Blockchains have many other applications outside of crypto, some of which are also awesome. **It just cannot be seen in isolation.**

And I think that makes for a pretty fitting conclusion to this experiment. Hope it was helpful to you!

(Feel free to drop me messages on how to code better or anything I've missed: You can find me at @whatislaweven on Instagram, whatislaweven@gmail.com, or "Darren Ang" on Linkedin!)

~

### N.B.: Some other things I decided not to include:

#### How can you be sure that the transacting parties are really who they are?

The private and public key system. These form a part of blocks' data, which generally gets hashed. On public and private keys generally, this blog post provides a great summary: https://www.preveil.com/blog/public-and-private-key/

#### Doesn't Bitcoin mining give you Bitcoin?

Yep, it does! To give its users incentives to run the proof-of-work mechanisms, Bitcoin gives a small amount of its currency to the **first** miner (more specifically, the human behind the miner bot) which solves the proof-of-work puzzle. 

This, of course, has become so lucrative that people are buying entire rigs of CPUs just to mine crypto. This has wasted tons of electricity, and created loads of wasteful heat that gets dissipated into the environment. So some cryptocurrencies, namely Ethereum, are exploring a new concept: 'proof-of-stake'. That might be my next project.